<a href="https://colab.research.google.com/github/ja-pa/ea-openai-evals/blob/main/OAI_evals_edit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Eval tests


In [1]:
#@title OpenAI API key


openai_key = '' #@param {type:"string"}

In [2]:
#@title Setup and dependecies

!git clone https://github.com/ja-pa/evals.git
!cd evals && git checkout match_example
!cd evals && git lfs fetch --all && git lfs pull
#!cat evals/evals/registry/data/logic/samples.jsonl
!cd evals && pip install -e .
!pip install -q gradio


import pandas as pd
import json
from google.colab import data_table

import os
import glob

import gradio as gr
import random

def display_table_from_file(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            # Parse each line as a JSON object
            json_line = json.loads(line)

            # Initialize content variables for different roles
            system_content = None
            user_content = None

            # Loop through 'input' list and extract 'content' for each 'role'
            for input_dict in json_line['input']:
                if input_dict['role'] == 'system':
                    system_content = input_dict['content']
                elif input_dict['role'] == 'user':
                    user_content = input_dict['content']

            # Extract 'ideal' field
            ideal = json_line['ideal']

            # Append to data list
            data.append({'System Content': system_content, 'User Content': user_content, 'Ideal': ideal})

    # Convert the list to a DataFrame
    df = pd.DataFrame(data)
    return df

def get_latest_file(path, pattern):
    list_of_files = glob.glob(os.path.join(path, pattern))
    if not list_of_files:
        return None
    latest_file = max(list_of_files, key=os.path.getctime)
    return latest_file


#log_output="/tmp/evallogs/230530205047OSMJXQ5A_gpt-3.5-turbo_logic-fact.jsonl"
def print_log(log_output):
  with open(log_output,"r") as fp:
    for line in fp.readlines():
      json_object = json.loads(line)
      json_formatted_str = json.dumps(json_object, indent=2)
      print(json_formatted_str)


def append_line_to_file(file_path, line):
    with open(file_path, 'a') as file:
        file.write(line + '\n')

# usage example

import pandas as pd
import json

def print_jsonl_table(filepath):
    # Read the jsonl file
    with open(filepath, 'r') as f:
        lines = f.readlines()

    # Parse each line as json
    data = [json.loads(line) for line in lines if line.strip()]

    # Merge data based on run_id and sample_id
    merged_data = {}
    for d in data:
        if 'run_id' in d and 'sample_id' in d:
            key = (d["run_id"], d["sample_id"])
            if key not in merged_data:
                merged_data[key] = d
            else:
                merged_data[key]['data'].update(d['data'])

    # Convert the merged data into a list of dictionaries
    data_list = list(merged_data.values())

    # Create a pandas dataframe from the data
    df = pd.DataFrame(data_list)

    # Explode the 'data' column into separate columns
    df = pd.concat([df.drop(['data'], axis=1), df['data'].apply(pd.Series)], axis=1)

    # Split 'prompt' into 'system' and 'user' columns
    df['system_prompt'] = df['prompt'].apply(lambda x: x[0]['content'] if x else None)
    df['user_prompt'] = df['prompt'].apply(lambda x: x[1]['content'] if x else None)
    df = df.drop('prompt', axis=1)

    # Drop unnecessary columns
    df = df.drop(['event_id', 'run_id', 'type', 'created_by'], axis=1)

    # Reorder columns
    df = df[['sample_id', 'created_at', 'system_prompt', 'user_prompt', 'expected', 'sampled', 'picked', 'correct']]

    # Display the dataframe
    display(df)


def add_custom_eval(description, eval, idea, output_file):
    textik='{"input": [{"role": "system", "content": "%s"}, {"role": "user", "content": "%s"}], "ideal": "%s"}'
    filled_textik=textik % (description,eval,idea)
    append_line_to_file("evals/evals/registry/data/test_match/samples.jsonl", filled_textik)
    return filled_textik

iface = gr.Interface(
    fn=add_custom_eval, 
    inputs=[gr.inputs.Textbox(lines=4, label="Content description"),
            gr.inputs.Textbox(lines=4, label="Eval"),
            gr.inputs.Textbox(lines=4, label="Ideal")
            ], 
    outputs=[gr.outputs.Textbox(label="Output Textbox 1")],
    description="Add content description, eval and ideal output"
)

#eval_name="test-match-custom"
#eval_dir_name="test_match"
#eval_sample_name=eva_dir_name+"/samples.jsonl"


Cloning into 'evals'...
remote: Enumerating objects: 2926, done.
remote: Counting objects: 100% (1332/1332), done.
remote: Compressing objects: 100% (272/272), done.
remote: Total 2926 (delta 1186), reused 1060 (delta 1060), pack-reused 1594
Receiving objects: 100% (2926/2926), 1.50 MiB | 7.94 MiB/s, done.
Resolving deltas: 100% (1554/1554), done.
Filtering content: 100% (258/258), 339.24 MiB | 45.16 MiB/s, done.
Branch 'match_example' set up to track remote branch 'match_example' from 'origin'.
Switched to a new branch 'match_example'
fetch: 273 object(s) found, done.
fetch: Fetching all references...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/evals
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/

/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:758: UserWarning: Expected 4 arguments for function <function add_custom_eval at 0x7fb9c3984af0>, received 3.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:762: UserWarn

# Eval
Add, show eval, run, show output log

In [3]:
#@title Eval add form

iface.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [7]:
#@title Show table with tested evals
#df = display_table_from_file('evals/evals/registry/data/test_match/samples.jsonl')
#display(df)
!cat evals/evals/registry/data/test_match/samples.jsonl
#!cat evals/evals/registry/data/logic/samples.jsonl

{"input": [{"role": "system", "content": "Complete the phrase as concisely as possible."}, {"role": "user", "content": "Once upon a "}], "ideal": "time"}
{"input": [{"role": "system", "content": "Complete the phrase as concisely as possible."}, {"role": "user", "content": "The first US president was "}], "ideal": "George Washington"}
{"input": [{"role": "system", "content": "Complete the phrase as concisely as possible."}, {"role": "user", "content": "OpenAI was founded in 20"}], "ideal": "15"}
{"input": [{"role": "system", "content": "Complete the phrase as concisely as possible"}, {"role": "user", "content": "Prague is capital of"}], "ideal": "Czech Republic"}


In [10]:
#@title Run eval

!cd evals && OPENAI_API_KEY=$openai_key oaieval gpt-3.5-turbo test-match-custom


[2023-06-06 16:32:22,883] [registry.py:250] Loading registry from /content/evals/evals/registry/evals
[2023-06-06 16:32:23,215] [registry.py:250] Loading registry from /root/.evals/evals
[2023-06-06 16:32:23,217] [oaieval.py:110] Run started: 2306061632232VOTIT6E
[2023-06-06 16:32:23,218] [data.py:75] Fetching test_match/samples.jsonl
[2023-06-06 16:32:23,218] [eval.py:33] Evaluating 5 samples
[2023-06-06 16:32:23,223] [eval.py:138] Running in threaded mode with 10 threads!
100% 5/5 [00:01<00:00,  3.34it/s]
[2023-06-06 16:32:24,729] [record.py:341] Final report: {'accuracy': 0.8}. Logged to /tmp/evallogs/2306061632232VOTIT6E_gpt-3.5-turbo_test-match-custom.jsonl
[2023-06-06 16:32:24,729] [oaieval.py:147] Final report:
[2023-06-06 16:32:24,729] [oaieval.py:149] accuracy: 0.8
[2023-06-06 16:32:24,731] [record.py:330] Logged 10 rows of events to /tmp/evallogs/2306061632232VOTIT6E_gpt-3.5-turbo_test-match-custom.jsonl: insert_time=0.992ms


In [11]:
#@title Show eval output log

# Call the function with your JSONL file path
#print_jsonl_table('your_file.jsonl')


log_output=get_latest_file("/tmp/evallogs", "*gpt-3.5-turbo_test-match-custom.jsonl")

print(log_output)
print_jsonl_table(log_output)


/tmp/evallogs/2306061632232VOTIT6E_gpt-3.5-turbo_test-match-custom.jsonl


,sample_id,created_at,system_prompt,user_prompt,expected,sampled,picked,correct
0,test-match-custom.s1.2,2023-06-06 16:32:23.965931+00:00,Complete the phrase as concisely as possible.,OpenAI was founded in 20,15,15.,15,True
1,test-match-custom.s1.1,2023-06-06 16:32:24.046020+00:00,Complete the phrase as concisely as possible.,The first US president was,George Washington,George Washington.,George Washington,True
2,test-match-custom.s1.0,2023-06-06 16:32:24.121877+00:00,Complete the phrase as concisely as possible.,Once upon a,time,time.,time,True
3,test-match-custom.s1.3,2023-06-06 16:32:24.345351+00:00,Complete the phrase as concisely as possible,Prague is capital of,Czech Republic,Czech Republic.,Czech Republic,True
4,test-match-custom.s1.4,2023-06-06 16:32:24.727033+00:00,Complete the phrase as concisely as possible,Prague is not,Brno,Prague is not Paris.,None,False
